🔗 [Back to Table of Contents](https://github.com/najaeda/najaeda-tutorials#-table-of-contents)
# Chapter 1: Getting Started with najaeda

Welcome to **najaeda**! This guide will help you quickly install and start using the `najaeda` Python package.

If you encounter any issues or have suggestions for improvements, please open an issue in our [GitHub issue tracker](https://github.com/najaeda/naja/issues).

If you find this project helpful, consider supporting us by [⭐️ starring the repository](https://github.com/najaeda/naja).

For inquiries or direct contact, feel free to reach out to us at [contact@keplertech.io](mailto:contact@keplertech.io).

## Introduction

**najaeda** is a powerful yet lightweight framework that empowers both software and hardware developers to efficiently explore, manipulate, and prototype electronic design automation (EDA) workflows.

Designed with flexibility and usability in mind, `najaeda` enables you to:

---

### Explore Netlists with Ease

- Intuitively navigate complex netlist hierarchies and connectivity.
- Inspect your design at various levels of abstraction:
  - Bit-level and bus-level granularity.
  - Instance-by-instance views or fully flattened representations down to primitives.
  - Focused per-instance connectivity or holistic equipotential (net-driven) views.

### Perform ECO (Engineering Change Order) Transformations

- Apply and manage design modifications smoothly.
- Accelerate the integration of late-stage design changes with confidence and control.

### Prototype EDA Concepts Rapidly

- Leverage an intuitive and expressive Python API.
- Experiment with new ideas, workflows, or data models in minutes, not days.

### Build Custom EDA Tools

- Develop high-performance, task-specific tools without depending on expensive, proprietary EDA suites.
- Solve domain-specific challenges with code that’s lean, readable, and maintainable.

## Installation

First, let's clone najaeda-tutorials repository to access the tutorials files.

In [ ]:
!git clone https://github.com/najaeda/najaeda-tutorials.git

To install `najaeda`, simply use pip:

In [2]:
pip install najaeda

Note: you may need to restart the kernel to use updated packages.


## Import najaeda
Now we will import najaeda needed package:

In [3]:
from najaeda import netlist

## Loading Primitives with Liberty

In [ ]:
liberty_files = [
    'najaeda-tutorials/benchmarks/liberty/fakeram45_64x32.lib',
    'najaeda-tutorials/benchmarks/liberty/fakeram45_1024x32.lib',
    'najaeda-tutorials/benchmarks/liberty/NangateOpenCellLibrary_typical.lib']
netlist.load_liberty(liberty_files)

## Load a verilog design
You can now load a gate level netlist using the loaded library.

In [ ]:
top = netlist.load_verilog(['najaeda-tutorials/benchmarks/verilog/tinyrocket.v'])

## Explore the design
We can now browse through the design.

In [ ]:
print(top)
print(f'{top} number of input terms: {top.count_input_terms()}')
print(f'{top} number of output terms: {top.count_output_terms()}')